In [85]:
import requests
import json
from math import ceil
import time
import psycopg2 as pg
import pandas as pd
import numpy as np
import io

In [10]:
API_KEY = 'S47aT37T3flUeMeR1-UisjDOYz1atYlD69BDiqvOhXTZb1dYdvDemGueu5vZxh5gvNWCx1OpOfXPLWmdAb-LiiI_EhU0nC8g8ES0sDGgV9H1FDhEl0PiDR_g5zPYZXYx'


In [325]:
engine = pg.connect("dbname='airflow' user='postgres' host='localhost' port='5432' password='admin'")
df = pd.read_sql('select * from common.zips', con=engine)

C:\Users\86136\AppData\Local\Temp\ipykernel_27428\1044340143.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('select * from common.zips', con=engine)


In [328]:
df_la = df[df.county == 'Los Angeles County']
df_la

,zip,city,state,county,country
38211,90001,Los Angeles,CA,Los Angeles County,US
38212,90002,Los Angeles,CA,Los Angeles County,US
38213,90003,Los Angeles,CA,Los Angeles County,US
38214,90004,Los Angeles,CA,Los Angeles County,US
38215,90005,Los Angeles,CA,Los Angeles County,US
...,...,...,...,...,...
39567,93584,Lancaster,CA,Los Angeles County,US
39568,93586,Lancaster,CA,Los Angeles County,US
39569,93590,Palmdale,CA,Los Angeles County,US
39570,93591,Palmdale,CA,Los Angeles County,US


In [ ]:
all_city_res = []

In [82]:

for city in df_la.city_ascii.unique().tolist()[3:]:
    print(city)
    city_string = f"{'%20'.join(city.split(' '))}%2C%20LA"
    
    url = f"https://api.yelp.com/v3/businesses/search?location={city_string}&sort_by=review_count&limit=50"

    headers = {
        "accept": "application/json",
        "Authorization": 'bearer %s' % API_KEY}
    
    
    response = requests.get(url = url, headers=headers)
    res= json.loads(response.text)
    total_res = res['total']
    all_res = [res]
    print('total number of food', total_res)

    iter_num = min(ceil(total_res/50 - 1), 19)
    print('total iterations',iter_num)
    error_res = []

    for i in range(1, iter_num):
        retry = 0
        PARAM = {'offset':i*50}

        url = f"https://api.yelp.com/v3/businesses/search?location={city_string}&sort_by=review_count&limit=50"

        headers = {
            "accept": "application/json",
            "Authorization": 'bearer %s' % API_KEY}

        response = requests.get(url = url, headers=headers, params=PARAM)
        while response.status_code != 200:
            if retry >= 5:
                error_res += [i]
                print('retries failed with ', i)
                print('-------------')
                break

            print('error with')
            time.sleep(10)
            response = requests.get(url = url, headers=headers, params=PARAM)
            retry+=1

        if response.status_code == 200:
            res= json.loads(response.text)
            print(res['businesses'][0]['name'])
            all_res.append(res)
            time.sleep(10)
        
    all_city_res.append(all_res)
    time.sleep(15)

Glendale
total number of food 5100
total iterations 19
Cafe Gratitude


KeyboardInterrupt: 

In [29]:
len(all_city_res)

2

In [39]:
all_city_res[0][0]['businesses'][0]

{'id': 'jkBSvGSIRnWrOTrGZR8rQw',
 'alias': 'gen-korean-bbq-house-cerritos',
 'name': 'Gen Korean BBQ House',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/w3Agb_BkWLNpClBInrK4SA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/gen-korean-bbq-house-cerritos?adjust_creative=EY-bgGJxBJ3rvqylo3Cj1w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=EY-bgGJxBJ3rvqylo3Cj1w',
 'review_count': 9010,
 'categories': [{'alias': 'korean', 'title': 'Korean'},
  {'alias': 'bbq', 'title': 'Barbeque'},
  {'alias': 'asianfusion', 'title': 'Asian Fusion'}],
 'rating': 4.3,
 'coordinates': {'latitude': 33.85729219286505, 'longitude': -118.088784},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '11472 S St',
  'address2': '',
  'address3': '',
  'city': 'Cerritos',
  'zip_code': '90703',
  'country': 'US',
  'state': 'CA',
  'display_address': ['11472 S St', 'Cerritos, CA 90703']},
 'phone': '+15629915411',
 'display_phone': '(562) 991-5411',
 'dis

In [ ]:
all_business = []

In [44]:
len(all_city_res[0])

18

In [79]:
final = []
for city_res in all_city_res:
    for res in city_res:
        final+=res['businesses']
    

In [80]:
len(final)

1850

In [215]:
final_clean = pd.DataFrame(final)
final_clean.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,jkBSvGSIRnWrOTrGZR8rQw,gen-korean-bbq-house-cerritos,Gen Korean BBQ House,https://s3-media2.fl.yelpcdn.com/bphoto/w3Agb_...,False,https://www.yelp.com/biz/gen-korean-bbq-house-...,9010,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.3,"{'latitude': 33.85729219286505, 'longitude': -...",[],$$,"{'address1': '11472 S St', 'address2': '', 'ad...",+15629915411,(562) 991-5411,9107.864553
1,tsJOEvDXaIG9IgbYJMaW5A,portos-bakery-and-cafe-downey,Porto's Bakery & Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/b8QoWD...,False,https://www.yelp.com/biz/portos-bakery-and-caf...,8573,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.4,"{'latitude': 33.94078, 'longitude': -118.13354}",[delivery],$,"{'address1': '8233 Firestone Blvd', 'address2'...",+15628628888,(562) 862-8888,15596.784890


In [115]:
final_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1850 entries, 0 to 1849
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1850 non-null   object 
 1   alias          1850 non-null   object 
 2   name           1850 non-null   object 
 3   image_url      1850 non-null   object 
 4   is_closed      1850 non-null   bool   
 5   url            1850 non-null   object 
 6   review_count   1850 non-null   int64  
 7   categories     1850 non-null   object 
 8   rating         1850 non-null   float64
 9   coordinates    1850 non-null   object 
 10  transactions   1850 non-null   object 
 11  price          1782 non-null   object 
 12  location       1850 non-null   object 
 13  phone          1850 non-null   object 
 14  display_phone  1850 non-null   object 
 15  distance       1850 non-null   float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 218.7+ KB


In [226]:
raw_data_create_SQL = '''
    TRUNCATE TABLE public.yelp_raw_data;
    DROP TABLE IF EXISTS public.yelp_raw_data;
    CREATE TABLE public.yelp_raw_data(
        id text,
        alias text,
        name text,
        image_url text,
        is_closed text,
        url text,
        review_count BIGINT,
        categories text,
        rating decimal,
        coordinates text,
        transactions text,
        price varchar(10),
        location text,
        phone text,
        display_phone text,
        distance decimal,
        PRIMARY KEY (id)
    )

'''

In [227]:
con = pg.connect(
   database="airflow",
   user="postgres",
   password="admin",
   host="localhost",
   port= '5432'
   )

cursor = con.cursor()


In [228]:
cursor.execute(raw_data_create_SQL)


In [229]:
output = io.StringIO()
final_clean = final_clean.drop_duplicates(subset = ['id'])
final_clean.to_csv(output, sep='\t', header=False, index=False)
output.seek(0)
contents = output.getvalue()

In [230]:
schema_name = 'public'
table_name = 'yelp_raw_data'

In [231]:
cursor.copy_expert(f"COPY {schema_name}.{table_name} FROM STDIN", output)


In [232]:
con.commit()
con.close()
cursor.close()

In [282]:
final_clean[['id', 'name', 'coordinates']]

,id,name,coordinates
0,jkBSvGSIRnWrOTrGZR8rQw,Gen Korean BBQ House,"{'latitude': 33.85729219286505, 'longitude': -..."
1,tsJOEvDXaIG9IgbYJMaW5A,Porto's Bakery & Cafe,"{'latitude': 33.94078, 'longitude': -118.13354}"
2,XpQIOGmb4YO14YUhQDUbQw,Porto's Bakery & Cafe,"{'latitude': 33.8524227, 'longitude': -117.997..."
3,lx3ex11xKnykVholglS8Vw,The Attic,"{'latitude': 33.764636, 'longitude': -118.151564}"
4,C6rP97tf5yi7INJCUH9A5w,Gen Korean BBQ House,"{'latitude': 33.80533, 'longitude': -118.32835}"
...,...,...,...
1845,f8OAZMH20W0PEwLj3VaKoQ,Luv'n Donuts,"{'latitude': 34.452224, 'longitude': -118.616627}"
1846,za17g7s1qU-R3-vcSmu8Fg,El Caporal Restaurant,"{'latitude': 34.3074635, 'longitude': -118.430..."
1847,I_ntPp1R6Pq3IHo14Wzd1w,Michexgod,"{'latitude': 34.3081, 'longitude': -118.4501}"
1848,d2HTwgx6RHGsk4ajksMAhg,Kogi BBQ Truck,"{'latitude': 34.2649073107067, 'longitude': -1..."


In [339]:
final_clean_location_df = pd.concat([final_clean.drop(columns=['coordinates']).reset_index(), pd.DataFrame(final_clean['coordinates'].values.tolist())], axis = 1).drop(columns=['index'])
final_clean_location_df.head(5)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude
0,jkBSvGSIRnWrOTrGZR8rQw,gen-korean-bbq-house-cerritos,Gen Korean BBQ House,https://s3-media2.fl.yelpcdn.com/bphoto/w3Agb_...,False,https://www.yelp.com/biz/gen-korean-bbq-house-...,9010,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.3,[],$$,"{'address1': '11472 S St', 'address2': '', 'ad...",+15629915411,(562) 991-5411,9107.864553,33.857292,-118.088784
1,tsJOEvDXaIG9IgbYJMaW5A,portos-bakery-and-cafe-downey,Porto's Bakery & Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/b8QoWD...,False,https://www.yelp.com/biz/portos-bakery-and-caf...,8573,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.4,[delivery],$,"{'address1': '8233 Firestone Blvd', 'address2'...",+15628628888,(562) 862-8888,15596.784890,33.940780,-118.133540
2,XpQIOGmb4YO14YUhQDUbQw,portos-bakery-and-cafe-buena-park-7,Porto's Bakery & Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/YwjBdh...,False,https://www.yelp.com/biz/portos-bakery-and-caf...,7448,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.4,[delivery],$,"{'address1': '7640 Beach Blvd', 'address2': ''...",+17143672030,(714) 367-2030,16190.819954,33.852423,-117.997423
3,lx3ex11xKnykVholglS8Vw,the-attic-long-beach,The Attic,https://s3-media1.fl.yelpcdn.com/bphoto/8emwss...,False,https://www.yelp.com/biz/the-attic-long-beach?...,7395,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.2,"[pickup, delivery]",$$,"{'address1': '3441 E Broadway', 'address2': ''...",+15624330153,(562) 433-0153,4319.654783,33.764636,-118.151564
4,C6rP97tf5yi7INJCUH9A5w,gen-korean-bbq-house-torrance-2,Gen Korean BBQ House,https://s3-media2.fl.yelpcdn.com/bphoto/gxfrfN...,False,https://www.yelp.com/biz/gen-korean-bbq-house-...,6864,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.1,[],$$,"{'address1': '24301 Crenshaw Blvd', 'address2'...",+14243280388,(424) 328-0388,15373.893050,33.805330,-118.328350


In [340]:
final_clean_location_df = pd.concat([final_clean_location_df.reset_index(), pd.DataFrame(final_clean_location_df['location'].values.tolist())[['zip_code', 'city', 'state', 'country']]], axis = 1).drop(columns=['index'])
final_clean_location_df.head(2) 

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,location,phone,display_phone,distance,latitude,longitude,zip_code,city,state,country
0,jkBSvGSIRnWrOTrGZR8rQw,gen-korean-bbq-house-cerritos,Gen Korean BBQ House,https://s3-media2.fl.yelpcdn.com/bphoto/w3Agb_...,False,https://www.yelp.com/biz/gen-korean-bbq-house-...,9010,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.3,[],...,"{'address1': '11472 S St', 'address2': '', 'ad...",+15629915411,(562) 991-5411,9107.864553,33.857292,-118.088784,90703,Cerritos,CA,US
1,tsJOEvDXaIG9IgbYJMaW5A,portos-bakery-and-cafe-downey,Porto's Bakery & Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/b8QoWD...,False,https://www.yelp.com/biz/portos-bakery-and-caf...,8573,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.4,[delivery],...,"{'address1': '8233 Firestone Blvd', 'address2'...",+15628628888,(562) 862-8888,15596.784890,33.940780,-118.133540,90241,Downey,CA,US


In [341]:
final_clean_location_df = pd.merge(left = final_clean_location_df, right = df_la[['zip', 'county']], left_on = 'zip_code', right_on = 'zip', how = 'left')
final_clean_location_df = final_clean_location_df.drop(columns = ['zip'])
final_clean_location_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,phone,display_phone,distance,latitude,longitude,zip_code,city,state,country,county
0,jkBSvGSIRnWrOTrGZR8rQw,gen-korean-bbq-house-cerritos,Gen Korean BBQ House,https://s3-media2.fl.yelpcdn.com/bphoto/w3Agb_...,False,https://www.yelp.com/biz/gen-korean-bbq-house-...,9010,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.3,[],...,+15629915411,(562) 991-5411,9107.864553,33.857292,-118.088784,90703,Cerritos,CA,US,Los Angeles County
1,tsJOEvDXaIG9IgbYJMaW5A,portos-bakery-and-cafe-downey,Porto's Bakery & Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/b8QoWD...,False,https://www.yelp.com/biz/portos-bakery-and-caf...,8573,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.4,[delivery],...,+15628628888,(562) 862-8888,15596.784890,33.940780,-118.133540,90241,Downey,CA,US,Los Angeles County
2,XpQIOGmb4YO14YUhQDUbQw,portos-bakery-and-cafe-buena-park-7,Porto's Bakery & Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/YwjBdh...,False,https://www.yelp.com/biz/portos-bakery-and-caf...,7448,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.4,[delivery],...,+17143672030,(714) 367-2030,16190.819954,33.852423,-117.997423,90620,Buena Park,CA,US,NaN
3,lx3ex11xKnykVholglS8Vw,the-attic-long-beach,The Attic,https://s3-media1.fl.yelpcdn.com/bphoto/8emwss...,False,https://www.yelp.com/biz/the-attic-long-beach?...,7395,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.2,"[pickup, delivery]",...,+15624330153,(562) 433-0153,4319.654783,33.764636,-118.151564,90803,Long Beach,CA,US,Los Angeles County
4,C6rP97tf5yi7INJCUH9A5w,gen-korean-bbq-house-torrance-2,Gen Korean BBQ House,https://s3-media2.fl.yelpcdn.com/bphoto/gxfrfN...,False,https://www.yelp.com/biz/gen-korean-bbq-house-...,6864,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.1,[],...,+14243280388,(424) 328-0388,15373.893050,33.805330,-118.328350,90505,Torrance,CA,US,Los Angeles County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,f8OAZMH20W0PEwLj3VaKoQ,luvn-donuts-castaic,Luv'n Donuts,https://s3-media4.fl.yelpcdn.com/bphoto/O84mvE...,False,https://www.yelp.com/biz/luvn-donuts-castaic?a...,81,"[{'alias': 'donuts', 'title': 'Donuts'}, {'ali...",3.5,"[delivery, pickup]",...,+16612572286,(661) 257-2286,9554.631110,34.452224,-118.616627,91384,Castaic,CA,US,Los Angeles County
1841,za17g7s1qU-R3-vcSmu8Fg,el-caporal-restaurant-sylmar,El Caporal Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/qFXpeL...,False,https://www.yelp.com/biz/el-caporal-restaurant...,81,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.9,[delivery],...,+18183626265,(818) 362-6265,14993.777522,34.307463,-118.430529,91342,Sylmar,CA,US,Los Angeles County
1842,I_ntPp1R6Pq3IHo14Wzd1w,michexgod-sylmar-2,Michexgod,https://s3-media3.fl.yelpcdn.com/bphoto/g0loGt...,False,https://www.yelp.com/biz/michexgod-sylmar-2?ad...,81,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",4.4,[],...,+18187448097,(818) 744-8097,14036.695672,34.308100,-118.450100,91342,Sylmar,CA,US,Los Angeles County
1843,d2HTwgx6RHGsk4ajksMAhg,kogi-bbq-truck-northridge,Kogi BBQ Truck,https://s3-media3.fl.yelpcdn.com/bphoto/0j_kAY...,False,https://www.yelp.com/biz/kogi-bbq-truck-northr...,81,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",3.9,[],...,,,17297.818218,34.264907,-118.521824,91311,Northridge,CA,US,Los Angeles County


In [349]:
final_clean_location_df[final_clean_location_df.county.isna()]

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,phone,display_phone,distance,latitude,longitude,zip_code,city,state,country,county
2,XpQIOGmb4YO14YUhQDUbQw,portos-bakery-and-cafe-buena-park-7,Porto's Bakery & Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/YwjBdh...,False,https://www.yelp.com/biz/portos-bakery-and-caf...,7448,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.4,[delivery],...,+17143672030,(714) 367-2030,16190.819954,33.852423,-117.997423,90620,Buena Park,CA,US,NaN
5,TqhuzJxO3A6jgf7JHGbBng,mamas-on-39-huntington-beach,Mama's On 39,https://s3-media2.fl.yelpcdn.com/bphoto/jMQy_h...,False,https://www.yelp.com/biz/mamas-on-39-huntingto...,5174,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,"[pickup, delivery]",...,+17143741166,(714) 374-1166,22855.048590,33.657432,-117.986997,92648,Huntington Beach,CA,US,NaN
7,uPWsnPg8M3ZRtGLPDP8JUw,bwon-shabu-and-bbq-fullerton-3,BWON Shabu & BBQ,https://s3-media2.fl.yelpcdn.com/bphoto/UGUpTe...,False,https://www.yelp.com/biz/bwon-shabu-and-bbq-fu...,5032,"[{'alias': 'hotpot', 'title': 'Hot Pot'}, {'al...",4.4,"[pickup, restaurant_reservation, delivery]",...,+17147700837,(714) 770-0837,19621.922640,33.860268,-117.961350,92833,Fullerton,CA,US,NaN
8,lSWHx8dI5yVyvfTpdrUpvQ,smoke-and-fire-social-eatery-la-habra-2,Smoke & Fire Social Eatery,https://s3-media3.fl.yelpcdn.com/bphoto/Q9CVV3...,False,https://www.yelp.com/biz/smoke-and-fire-social...,4945,"[{'alias': 'comfortfood', 'title': 'Comfort Fo...",4.7,"[pickup, delivery]",...,+15623159929,(562) 315-9929,24073.294695,33.940246,-117.960978,90631,La Habra,CA,US,NaN
9,Xo9gKql3lRZc2iRQcRwPtQ,the-donuttery-huntington-beach,The Donuttery,https://s3-media3.fl.yelpcdn.com/bphoto/F53KsP...,False,https://www.yelp.com/biz/the-donuttery-hunting...,4836,"[{'alias': 'donuts', 'title': 'Donuts'}, {'ali...",4.4,"[pickup, delivery]",...,+17148477000,(714) 847-7000,19083.570402,33.709461,-117.988621,92647,Huntington Beach,CA,US,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,mhQA8e1gzSJoOvUeIQAW0Q,jersey-mikes-subs-simi-valley-4,Jersey Mike's Subs,https://s3-media2.fl.yelpcdn.com/bphoto/VGWV24...,False,https://www.yelp.com/biz/jersey-mikes-subs-sim...,91,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.2,"[delivery, pickup]",...,+18055797808,(805) 579-7808,23367.747492,34.286788,-118.716240,93063,Simi Valley,CA,US,NaN
1802,o102G_PBTfjIxuNBrJz0fQ,dominos-pizza-simi-valley-3,Domino's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/ISRPk1...,False,https://www.yelp.com/biz/dominos-pizza-simi-va...,90,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.7,[],...,+18055229022,(805) 522-9022,22178.202203,34.279674,-118.690765,93063,Simi Valley,CA,US,NaN
1806,zH7B3ZzO0pegvfe4VhE1Yg,ameci-pizza-and-pasta-simi-valley-2,Ameci Pizza & Pasta,https://s3-media2.fl.yelpcdn.com/bphoto/4uGeec...,False,https://www.yelp.com/biz/ameci-pizza-and-pasta...,90,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"[delivery, pickup]",...,+18055819770,(805) 581-9770,23489.899459,34.285810,-118.717080,93063,Simi Valley,CA,US,NaN
1835,8qd6Ox8eDxeaaCCdDYNNjQ,cold-stone-creamery-simi-valley,Cold Stone Creamery,https://s3-media1.fl.yelpcdn.com/bphoto/DKxgeQ...,False,https://www.yelp.com/biz/cold-stone-creamery-s...,82,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",2.9,"[delivery, pickup]",...,+18053061184,(805) 306-1184,23417.026227,34.287567,-118.717535,93063,Simi Valley,CA,US,NaN


In [342]:
clean_data_create_SQL = '''
    DROP TABLE IF EXISTS public.yelp_processed_data;
    CREATE TABLE public.yelp_processed_data(
        id text,
        alias text,
        name text,
        image_url text,
        is_closed text,
        url text,
        review_count BIGINT,
        categories text,
        rating decimal,
        transactions text,
        price varchar(10),
        location text,
        phone text,
        display_phone text,
        distance decimal,
        latitude decimal,
        longitude decimal,
        zip_code bigint,
        city text,
        state varchar(2),
        country varchar(5),
        county text,
        PRIMARY KEY (id)
    )

'''

In [343]:
con = pg.connect(
   database="airflow",
   user="postgres",
   password="admin",
   host="localhost",
   port= '5432'
   )

cursor = con.cursor()


In [344]:
cursor.execute(clean_data_create_SQL)


In [345]:
output = io.StringIO()
final_clean_location_df = final_clean_location_df.drop_duplicates(subset = ['id'])
final_clean_location_df.to_csv(output, sep='\t', header=False, index=False)
output.seek(0)
contents = output.getvalue()

In [346]:
schema_name = 'public'
table_name = 'yelp_processed_data'

In [347]:
cursor.copy_expert(f"COPY {schema_name}.{table_name} FROM STDIN", output)


In [348]:
con.commit()
con.close()
cursor.close()

In [242]:
final_clean[['id', 'categories']]

,id,categories
0,jkBSvGSIRnWrOTrGZR8rQw,"[{'alias': 'korean', 'title': 'Korean'}, {'ali..."
1,tsJOEvDXaIG9IgbYJMaW5A,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {..."
2,XpQIOGmb4YO14YUhQDUbQw,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {..."
3,lx3ex11xKnykVholglS8Vw,"[{'alias': 'breakfast_brunch', 'title': 'Break..."
4,C6rP97tf5yi7INJCUH9A5w,"[{'alias': 'korean', 'title': 'Korean'}, {'ali..."
...,...,...
1845,f8OAZMH20W0PEwLj3VaKoQ,"[{'alias': 'donuts', 'title': 'Donuts'}, {'ali..."
1846,za17g7s1qU-R3-vcSmu8Fg,"[{'alias': 'mexican', 'title': 'Mexican'}]"
1847,I_ntPp1R6Pq3IHo14Wzd1w,"[{'alias': 'foodtrucks', 'title': 'Food Trucks..."
1848,d2HTwgx6RHGsk4ajksMAhg,"[{'alias': 'korean', 'title': 'Korean'}, {'ali..."


In [243]:
categories_df = final_clean[['id', 'categories']].explode('categories')
categories_df

,id,categories
0,jkBSvGSIRnWrOTrGZR8rQw,"{'alias': 'korean', 'title': 'Korean'}"
0,jkBSvGSIRnWrOTrGZR8rQw,"{'alias': 'bbq', 'title': 'Barbeque'}"
0,jkBSvGSIRnWrOTrGZR8rQw,"{'alias': 'asianfusion', 'title': 'Asian Fusion'}"
1,tsJOEvDXaIG9IgbYJMaW5A,"{'alias': 'bakeries', 'title': 'Bakeries'}"
1,tsJOEvDXaIG9IgbYJMaW5A,"{'alias': 'customcakes', 'title': 'Custom Cakes'}"
...,...,...
1847,I_ntPp1R6Pq3IHo14Wzd1w,"{'alias': 'seafood', 'title': 'Seafood'}"
1847,I_ntPp1R6Pq3IHo14Wzd1w,"{'alias': 'mexican', 'title': 'Mexican'}"
1848,d2HTwgx6RHGsk4ajksMAhg,"{'alias': 'korean', 'title': 'Korean'}"
1848,d2HTwgx6RHGsk4ajksMAhg,"{'alias': 'mexican', 'title': 'Mexican'}"


In [259]:
pd.DataFrame(categories_df['categories'].values.tolist())

,alias,title
0,korean,Korean
1,bbq,Barbeque
2,asianfusion,Asian Fusion
3,bakeries,Bakeries
4,customcakes,Custom Cakes
...,...,...
4203,seafood,Seafood
4204,mexican,Mexican
4205,korean,Korean
4206,mexican,Mexican


In [263]:
categories_final_df = pd.concat([categories_df[['id']].reset_index(), pd.DataFrame(categories_df['categories'].values.tolist())], axis = 1)
categories_final_df = categories_final_df.drop(columns = ['index'])
categories_final_df

,id,alias,title
0,jkBSvGSIRnWrOTrGZR8rQw,korean,Korean
1,jkBSvGSIRnWrOTrGZR8rQw,bbq,Barbeque
2,jkBSvGSIRnWrOTrGZR8rQw,asianfusion,Asian Fusion
3,tsJOEvDXaIG9IgbYJMaW5A,bakeries,Bakeries
4,tsJOEvDXaIG9IgbYJMaW5A,customcakes,Custom Cakes
...,...,...,...
4203,I_ntPp1R6Pq3IHo14Wzd1w,seafood,Seafood
4204,I_ntPp1R6Pq3IHo14Wzd1w,mexican,Mexican
4205,d2HTwgx6RHGsk4ajksMAhg,korean,Korean
4206,d2HTwgx6RHGsk4ajksMAhg,mexican,Mexican


In [270]:
category_data_create_SQL = '''
    DROP TABLE IF EXISTS public.yelp_category;
    CREATE TABLE public.yelp_category(
        num_id SERIAL PRIMARY KEY,
        id text,
        alias text,
        title text,
        CONSTRAINT fk_id
            FOREIGN KEY(id) 
            REFERENCES public.yelp_raw_data(id)
    )

'''

In [277]:
con = pg.connect(
   database="airflow",
   user="postgres",
   password="admin",
   host="localhost",
   port= '5432'
   )

cursor = con.cursor()


In [278]:
cursor.execute(category_data_create_SQL)


In [279]:
output = io.StringIO()
categories_final_df.to_csv(output, sep='\t', header=False, index=True)
output.seek(0)
contents = output.getvalue()
schema_name = 'public'
table_name = 'yelp_category'

In [280]:
cursor.copy_expert(f"COPY {schema_name}.{table_name} FROM STDIN", output)


In [281]:
con.commit()
con.close()
cursor.close()

In [ ]:
categories_df = final_clean[['id', 'categories']].explode('categories')
categories_df